In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
import catboost

In [18]:
mishafeat2 = pd.read_csv('new_features.csv')

In [19]:
del mishafeat2['Unnamed: 0']

In [39]:
X_all = X_all.merge(mishafeat2,how='outer', left_index=True, right_index=True)

In [20]:
mishafeat = pd.read_csv('magic.csv')

In [21]:
del mishafeat['Unnamed: 0']

In [22]:
def school_type(s_info):
    order = 'НВ СР СП ОСН ОО НП НС'
    def map_school(s: str):
        if s is np.nan:
            return s
        s = s.lower()
        if s == 'ИНОГОРОДНЯЯ СРЕДНЯЯ ШКОЛА'.lower():
            return s
        schools = {
            'средняя': 'средняя школа', 
            'сош': 'средняя школа',
            'сш': 'средняя школа',
            'вечерняя': 'вечерняя школа',
            'гимназия': None,
            'лицей': None,
            'коррекц': 'коррекционная школа',
            'интернат': 'школа-интернат',
            'колледж': None,
            'техникум': None,
            'училище': None,
            'пу': 'училище',
            'пту': 'училище',
            'университет': None,
            'академия': None,
        }
        for i in schools.items():
            if i[0] in s:
                if i[1]:
                    return i[1]
                else:
                    return i[0]
        return s + '!'
    
    s_info_dobr = s_info.copy()
    s_info_dobr['Образование'] = s_info['Образование'].replace({s: i for i, s in  enumerate(order.split() + [np.nan])})
    s_info_dobr =  s_info_dobr.sort_values('Образование').drop_duplicates(subset=['STD_ID'])
    s_info_dobr['Образование'] = s_info['Образование'].replace({i: s for i, s in  enumerate(order.split() + [np.nan])})
    s_info_dobr['Вид ОО'] = s_info_dobr['Что именно  закончил'].map(map_school)
    return s_info_dobr

In [23]:
def reduce_mem_usage(df, verbose=True):
    numerics = ["int16", "int32", "int64", "float16", "float32",  "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max <  np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max <  np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max <  np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df

In [24]:
from sklearn.model_selection import cross_val_score
def crossv(model, splits, x, y):
    kfoldd = KFold(n_splits=splits, random_state=7)
    results = cross_val_score(model, x, y, cv=kfoldd)
    return results

In [25]:
def encodew(w):
    try:
        parsed = morph.parse(w)
        return vecmodel.wv[parsed[0].normal_form+'_'+parsed[0].tag.POS]
    except:
        return None

In [26]:
def dropnn(mass):
    newmass = []
    for k in mass:
        if k is not None:
            newmass.append(k)
    return newmass

In [27]:
def vec_mean(vec):
    if len(vec) == 0:
        return vec
    elif len(vec) == 1:
        return vec[0]
    else:
        return np.ptp(vec, axis=0)

In [28]:
def enccateg(df):
    for col in df.columns:
        if df[col].dtype == 'object':
            le = preprocessing.LabelEncoder()
            le.fit(df[col])
            df[col] = le.transform(df[col])
    return df

# Обработка studs_info

In [29]:
studs_info = pd.read_csv("studs_info.csv")
studs_info = school_type(studs_info )
studs_info = studs_info.set_index('STD_ID')

In [30]:
def encodestudsinfo():
    global studs_info
    
    shif = studs_info['Шифр'].str.split('.', expand = True)
    for i, var in enumerate(shif):
        studs_info['ШИФР_'+str(i)] = shif[i]
    dates = ['year','month','day']
    dat = studs_info['Дата выпуска'].str.split('-', expand = True)
    for i, var in enumerate(dat):
        studs_info[dates[i]] = dat[i]
    dat = studs_info['Дата выдачи'].str.split('-', expand = True)
    for i, var in enumerate(dat):
        studs_info['Дата выдачи_'+dates[i]] = dat[i]
    studs_info['Дата выдачи_day'] = studs_info['Дата выдачи_day'].str.split('  ',expand=True)[0]
    studs_info = studs_info.loc[~studs_info.index.duplicated(keep='first')]
    studs_info = studs_info.rename(columns={'   ':'secret'})
    #for col in ['year','month','day','Пол','Статус', 'Дата выпуска', 'Категория  обучения', 'Форма обучения', 'направление  (специальность)','Образование','Дата выдачи','Что именно  закончил','Шифр','ШИФР_0','ШИФР_1','ШИФР_2']:
    #    studs_info[col] = studs_info[col].astype('str')
    #    le = preprocessing.LabelEncoder()
    #    print(studs_info[col])
    #    le.fit(studs_info[col])
    #    studs_info[col+'_enc'] = le.transform(studs_info[col])
    

In [31]:
encodestudsinfo()

# Склеивание всех сетов в один X_all

In [32]:
X_train_in = pd.read_csv("X_train.csv", index_col=0)
y_train_in = pd.read_csv("y_train.csv",index_col=0)
X_val = pd.read_csv("X_test.csv", index_col=0)
zachety = pd.read_csv('zachety.csv', index_col=0)

In [33]:
X_train_in['part'] = 'train'
X_val['part'] = 'val'
zachety['part'] = 'val2'

In [34]:
X_train_in["mark"]=y_train_in["mark"]
X_val['mark'] = np.nan
zachety['mark'] = np.nan

In [35]:
X_all = X_train_in.append(X_val, ignore_index=True)

In [36]:
X_all['magic'] = mishafeat

In [37]:
zachety['magic'] = np.nan

In [38]:
X_all = X_all.append(zachety, ignore_index=True)

In [35]:
X_all.head()

,STD_ID,НАПРАВЛЕНИЕ,ГОД,КУРС,СЕМЕСТР,АТТЕСТАЦИЯ,ДИСЦИПЛИНА,part,mark,magic,age,is_half_year,study_finished,date
0,175711,38.03.02 Менеджмент,2018-2019,2.0,4.0,Экзамен,Маркетинг,train,5.0,0.930184,20.0,0.0,0.500000,2018-2019 0
1,100647,31.05.01 Лечебное дело,2015-2016,3.0,5.0,Экзамен,"Пропедевтика внутренних болезней, лучевая диаг...",train,5.0,1.502872,20.5,1.0,0.416667,2015-2016 1
2,199311,31.05.03 Стоматология,2019-2020,1.0,1.0,Экзамен,Химия,train,5.0,1.290966,18.5,1.0,0.125000,2019-2020 1
3,162002,38.03.02 Менеджмент,2017-2018,2.0,3.0,Экзамен,Статистика,train,4.0,1.288821,19.5,1.0,0.375000,2017-2018 1
4,108384,31.05.01 Лечебное дело,2018-2019,5.0,9.0,Экзамен,Педиатрия,train,3.0,1.518378,22.5,1.0,0.750000,2018-2019 1


# СТАНДАРТНЫЕ ЕНКОДИНГИ 

In [40]:
from sklearn import preprocessing
def encode_cat():
    global X_all
    X_all['КУРС+СЕМЕСТР'] = X_all['КУРС'].astype('str') +  X_all['СЕМЕСТР'].astype('str')
    X_all['minist'] = X_all['НАПРАВЛЕНИЕ'].str.split(' ', expand = True)[0].str.split('.',    expand = True)[0]
    X_all['level'] = X_all['НАПРАВЛЕНИЕ'].str.split(' ',  expand = True)[0].str.split('.',   expand = True)[1]
    X_all['dirgroup'] = X_all['НАПРАВЛЕНИЕ'].str.split(' ',  expand =  True)[0].str.split('.',  expand = True)[2]
    X_all['direction'] = X_all['НАПРАВЛЕНИЕ'].str.split(' ',  expand = True)[1]
    #for col in ['НАПРАВЛЕНИЕ','ГОД', 'АТТЕСТАЦИЯ', 'ДИСЦИПЛИНА', 'direction',  'minist','level','dirgroup' ]:
    #    le = preprocessing.LabelEncoder()
    #    le.fit(X_all[col])
    #    X_all.loc[:,col+'_enc'] = le.transform(X_all.loc[:,col])

In [41]:
encode_cat()

In [42]:
def feat_gen():
    global X_all
    kf = KFold(n_splits=2, shuffle=True, random_state=None)
    for group in X_all.groupby("STD_ID")["mark"]:
        if group[1].size > 2:
            for train_index, test_index in kf.split(group[1]):
                aggs = group[1].iloc[train_index].agg(['mean'])
                for aggname in ['mean']:
                    X_all.loc[group[1].iloc[test_index].index, 'test_'+aggname] =    aggs[aggname]

In [44]:
def mormean():
    global X_all
    aggs = X_train_in.groupby("STD_ID")['mark'].agg(['mean'])
    for agg in ['mean']:
        X_all.loc[:, 'stud_'+agg] = X_all.loc[:,'STD_ID'].map(aggs[agg])

In [45]:
mormean()

# неудачная попытка сделать енкодинги

In [29]:
def specificbeh():
    global X_all
    for col in ['АТТЕСТАЦИЯ']:
        print(col)
        for group in X_all.groupby("STD_ID"):
            print(group[0], 'process')
            if len(group[1]) > 1:
                if len(group[1]) > 60:
                    splits = len(group[1]) // 10
                elif len(group[1]) > 45:
                    splits = len(group[1]) // 8
                elif len(group[1]) > 30:
                    splits = len(group[1]) // 5
                elif len(group[1]) > 12:
                    splits = len(group[1]) // 2
                else:
                    splits = len(group[1])
                kf = KFold(n_splits=splits, shuffle=True, random_state=None)
                for train_index, test_index in kf.split(group[1]):
                    colaggs = group[1].iloc[train_index].groupby(col)['mark'].mean()
                    for c in colaggs.index:
                        X_all.loc[group[1].iloc[test_index].index,  'stud_mean_specif_'+str(c)] = colaggs[c]
                    #colaggs = pd.DataFrame(X_train_in.groupby(["STD_ID",  col])['mark'].mean()).reset_index().pivot(index='STD_ID',  columns=col,values='mark')
                    #colaggs = colaggs.rename(columns={i:col+'_mean_'+str(i) for i in  colaggs.columns})
                    #for c in colaggs.columns:
                    #    X_all.loc[:, 'stud_mean_'+str(c)] =  X_all.loc[:,'STD_ID'].map(colaggs[c])'''

In [24]:
def splitintowords(s):
    return [i for i in s.replace('-',' ').replace(', ',' ').replace(' и ',' ').replace(':','  ').replace('(',' ').replace(')',' ').split(' ') if i != ' ' and i != '']

In [25]:
def normwords(arr):
    return [morph.parse(a)[0].normal_form for a in arr]

In [26]:
def vecthewords(arr):
    return [encodew(a) for a in arr]

# Попытка сделать кластеризацию по векторам Дисциплин (неудачная)

In [30]:
from sklearn.neighbors import KNeighborsClassifier

In [31]:
from sklearn.cluster import KMeans

In [36]:
word2vec = {}
for i in X_all.ДИСЦИПЛИНА.unique():
    r = vec_mean(dropnn(vecthewords(splitintowords(i))))
    if len(r) != 0:
        word2vec[i] = r
    else:
        print(i,normwords(splitintowords(i)))

/home/ec2-user/anaconda3/lib/python3.7/site- packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `wv`  (Attribute will be removed in 4.0.0, use self instead).
  after removing the cwd from sys.path.


Фтизиатрия ['фтизиатрия']
Патофизиология, клиническая патофизиология ['патофизиология',    'клинический', 'патофизиология']
Фармакогнозия ['фармакогнозия']
Бухгалтерская (финансовая) отчетность ['бухгалтерский', 'финансовый',  'отчётность']
Тепломассообмен ['тепломассообмен']
Бухгалтерский финансовый учет ['бухгалтерский', 'финансовый', 'учёт']
Логопедия ['логопедия']
Эконометрика ['эконометрика']
Бухгалтерский учет ['бухгалтерский', 'учёт']
Гидробиология ['гидробиология']
Мастерские (PR) ['мастерский', 'pr']
Лингвострановедение и страноведение ['лингвострановедение',  'страноведение']
Медиалингвистика ['медиалингвистика']
Налоговый учет ['налоговый', 'учёт']
Бухгалтерский управленческий учет ['бухгалтерский', 'управленческий',  'учёт']
Фотографика ['фотографик']
Лесоэксплуатация ['лесоэксплуатация']
Медицинское и фармацевтическое товароведение ['медицинский',  'фармацевтический', 'товароведение']
Фотодело ['фотодело']
Кормопроизводство ['кормопроизводство']
Типографика ['типографик']


In [37]:
X = word2vec.values()
X = list(X)

In [38]:
for cl in range(8,9):
    kmeans = KMeans(n_clusters=cl)
    kmeans.fit(X)
    preds = kmeans.predict(X)
    lol = {}
    for i, p in enumerate(preds):
        lol[list(word2vec.keys())[i]] = p
    X_all['ДИСЦИПЛИНА_smallgroup_'+str(cl)] =  X_all.ДИСЦИПЛИНА.apply(lambda x:lol[x] if x in lol else np.nan)

# векторизация дисциплин и направлений и новые фичи на базе векторов

In [46]:
import gensim, pymorphy2

In [47]:
vecmodel =  gensim.models.KeyedVectors.load_word2vec_format("model2/model.bin",  binary=True)
vecmodel.init_sims(replace=True)
morph = pymorphy2.MorphAnalyzer() 

In [48]:
for col in studs_info.columns:
    X_all.loc[:, 'studinf_'+col] = X_all.loc[:,'STD_ID'].map(studs_info[col])

In [49]:
def preprocessforenc():
    global X_all
    X_all['НАПРАВЛЕНИЕ'] = X_all['НАПРАВЛЕНИЕ'].str.split(' ',  expand=True)[1].fillna('') + ' ' + X_all['НАПРАВЛЕНИЕ'].str.split(' ',  expand=True)[2].fillna('') +' ' + X_all['НАПРАВЛЕНИЕ'].str.split(' ',  expand=True)[3].fillna('') +' ' + X_all['НАПРАВЛЕНИЕ'].str.split(' ',  expand=True)[4].fillna('') +' ' + X_all['НАПРАВЛЕНИЕ'].str.split(' ',  expand=True)[5].fillna('') +' ' + X_all['НАПРАВЛЕНИЕ'].str.split(' ',  expand=True)[6].fillna('')
    X_all['studinf_Дата выдачи'] = X_all['studinf_Дата выдачи'].str.split('  ',expand=True)[0]
    X_all['studinf_Дата выдачи_year'] = X_all['studinf_Дата выдачи'].str.split('- ',expand=True)[0]
    X_all['studinf_Дата выдачи_month'] = X_all['studinf_Дата выдачи'].str.split('- ',expand=True)[1]
    X_all['studinf_Дата выдачи_day'] = X_all['studinf_Дата выдачи'].str.split('- ',expand=True)[2]
    '''
    X_all.loc[X_all.ДИСЦИПЛИНА == 'Фтизиатрия', 'ДИСЦИПЛИНА'] = 'медицина  туберкулёз'
    X_all.loc[X_all.ДИСЦИПЛИНА == 'Педиатрия', 'ДИСЦИПЛИНА'] =  'педиатр'
    X_all.loc[X_all.ДИСЦИПЛИНА == 'Лесоустройство', 'ДИСЦИПЛИНА'] = 'лес  эколог'
    X_all.loc[X_all.ДИСЦИПЛИНА == 'Травматология, ортопедия',  'ДИСЦИПЛИНА'] = 'травма врач невропатолог'
    X_all.loc[X_all.ДИСЦИПЛИНА == 'Фармакогнозия', 'ДИСЦИПЛИНА'] =  'фармацевт лекарства'
    X_all.loc[X_all.ДИСЦИПЛИНА == 'Геоморфология и гидрология',  'ДИСЦИПЛИНА'] = 'геология география рельеф'
    X_all.loc[X_all.ДИСЦИПЛИНА == 'Офтальмология', 'ДИСЦИПЛИНА'] =  'офтальмолог'
    X_all.loc[X_all.ДИСЦИПЛИНА == 'Тепломассообмен', 'ДИСЦИПЛИНА'] =  'физика теплообмен'
    X_all.loc[X_all.ДИСЦИПЛИНА == 'Теплотехника', 'ДИСЦИПЛИНА'] = 'техника  физика инженер'
    X_all.loc[X_all.ДИСЦИПЛИНА == 'Эконометрика', 'ДИСЦИПЛИНА'] =  'экономика математика статистика'
    X_all.loc[X_all.ДИСЦИПЛИНА == 'Машиноведение', 'ДИСЦИПЛИНА'] =  'машина машинист'
    X_all.loc[X_all.НАПРАВЛЕНИЕ == 'Агрономия     ','НАПРАВЛЕНИЕ'] =  'ботаника'
    X_all.НАПРАВЛЕНИЕ[X_all.НАПРАВЛЕНИЕ == 'Педагогическое образование  (с двумя профилями подготовки)'] = 'Педагогическое образование' '''

In [50]:
preprocessforenc()

In [51]:
def wordvec():
    global X_all
    test = X_all.НАПРАВЛЕНИЕ.apply(lambda x:vec_mean(dropnn([encodew(i) for i  in x.replace('-',' ').replace(' и ',' ').replace('(',' ').replace(')',' ').split(' ') if i != ' ' and i !=  ''])) )
    test = pd.DataFrame([row for row in test], columns=['направление_enc'+str(i)    for i in range(300)])
    test.index = X_all.index
    X_all = X_all.merge(test,how='outer', left_index=True, right_index=True)
    test2 = X_all.ДИСЦИПЛИНА.apply(lambda x:vec_mean(dropnn([encodew(i) for  i in x.replace('-',' ').replace(', ',' ').replace(' и ',' ').replace('(',' ').replace(')',' ').split(' ') if  i != ' ' and i != ''])) )
    test2 = pd.DataFrame([row for row in test2],  columns=['направление_enc'+str(i) for i in range(300)])
    test2.index = X_all.index
    X_all = X_all.merge(test2,how='outer', left_index=True, right_index=True)
    meanwordvec = X_all.groupby('STD_ID')[['направление_enc{}_y'.format(str(i))  for i in  range(300)]].mean().rename(columns={'направление_enc{}_y'.format(str(i)):'напра вление_meanenc{}_y'.format(str(i)) for i in range(300)})
    for feat in meanwordvec.columns:
        X_all[feat] = X_all.loc[:,'STD_ID'].map(meanwordvec[feat])


In [52]:
wordvec()

/home/ec2-user/anaconda3/lib/python3.7/site- packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `wv`  (Attribute will be removed in 4.0.0, use self instead).
  after removing the cwd from sys.path.


In [53]:
def gen_feat_onenc():
    global X_all
    sqdiff = np.power(np.array(X_all[['направление_enc{}_y'.format(str(i)) for i in  range(300)]]) - np.array(X_all[['направление_meanenc{}_y'.format(str(i)) for i in  range(300)]]),2)
    sqdiff = np.sum(sqdiff, axis=1)
    X_all['encoding_diff'] = sqdiff
    sqdiffx = np.power(np.array(X_all[['направление_enc{}_y'.format(str(i)) for i in  range(300)]]) - np.array(X_all[['направление_enc{}_x'.format(str(i)) for i in  range(300)]]),2)
    sqdiffx = np.sum(sqdiffx, axis=1)
    X_all['encoding_diffnaprav'] = sqdiffx

In [54]:
gen_feat_onenc()

In [55]:
X_all['mark'] = X_all['mark'].astype(float)

# частота появления определенной категории по отношению к общему числу  для конкретного студента

In [56]:
def categ_ratio():
    global X_all
    for col in ['ГОД', 'КУРС','СЕМЕСТР','АТТЕСТАЦИЯ']:
        freq = X_all.groupby('STD_ID').apply(lambda  x:pd.DataFrame(x[col].value_counts()/len(x)).T)
        freq = freq.reset_index().drop(columns=['level_1']).set_index('STD_ID')
        freq.columns = [str(i)+'_freq' for i in freq.columns]
        for coll in freq.columns:
            freqq = X_all.loc[:,'STD_ID'].map(freq[coll])
            X_all[col+'_'+coll] = freqq

In [57]:
categ_ratio()

#  mean encoding по категориальным фичам с использованием KFold

In [58]:
def mean_encode():
    for col in ['НАПРАВЛЕНИЕ','ГОД', 'КУРС','СЕМЕСТР','studinf_Вид  ОО','АТТЕСТАЦИЯ','studinf_Дата выпуска','studinf_Категория  обучения','studinf_Форма обучения','studinf_Образование','studinf_Что именно  закончил','studinf_year','studinf_Дата  выдачи_year','studinf_Пол','studinf_Статус','minist', 'level',  'dirgroup','studinf_month','studinf_Дата выдачи_month']:
        if 'meanenc_'+col not in X_all.columns:
            kf = KFold(n_splits=4, shuffle=True, random_state=None)
            for train_index, test_index in kf.split(X_all):
                X_all.loc[test_index, 'meanenc_'+col] =  X_all.loc[test_index].loc[:,col].map(X_all.loc[train_index].groupby(col)['mark'].mean( ))
               
                #X_all.loc[:, 'meanenc_'+col] =  X_all.loc[:,col].map(X_all.groupby(col)['mark'].mean())
                
                

In [59]:
def std_encode():
    for col in ['НАПРАВЛЕНИЕ','ГОД', 'КУРС','СЕМЕСТР','studinf_Вид  ОО','АТТЕСТАЦИЯ','studinf_Дата выпуска','studinf_Категория  обучения','studinf_Форма обучения','studinf_Образование','studinf_Что именно  закончил','studinf_year','studinf_Дата  выдачи_year','studinf_Пол','studinf_Статус','minist', 'level',  'dirgroup','studinf_month','studinf_Дата выдачи_month']:
        if 'stdenc_'+col not in X_all.columns:
            kf = KFold(n_splits=4, shuffle=True, random_state=None)
            for train_index, test_index in kf.split(X_all):
                X_all.loc[test_index, 'stdenc_'+col] =  X_all.loc[test_index].loc[:,col].map(X_all.loc[train_index].groupby(col)['mark'].std())\ n                #X_all.loc[:, 'meanenc_'+col] =  X_all.loc[:,col].map(X_all.groupby(col)['mark'].mean())
                

In [60]:
def mean_encodedouble():
    global X_all
    for col1 in range(len(ftrs)):
        for col2 in range(col1+1, len(ftrs)):
            if ftrs[col1]+ftrs[col2]+"_meanencdouble" not in X_all.columns:
                kf = KFold(n_splits=4, shuffle=True, random_state=None)
                for train_index, test_index in kf.split(X_all):
                    groups = X_all.loc[train_index].groupby([ftrs[col1], ftrs[col2]])
                    X_all.loc[test_index, ftrs[col1]+ftrs[col2]+"_meanencdouble"] =  pd.MultiIndex.from_frame(X_all.loc[test_index].loc[:,[ftrs[col1],ftrs[col2]]]).map(gro ups['mark'].mean())


In [61]:
mean_encode()   

In [62]:
std_encode()

In [59]:
ftrs = ['НАПРАВЛЕНИЕ','studinf_Что именно закончил']
mean_encodedouble()

In [60]:
ftrs = ['НАПРАВЛЕНИЕ','studinf_Пол']
mean_encodedouble()

# Разделение X_all на X_val и X_train

In [63]:
colsss = list(X_all.columns)
colsss.remove('part')
colsss.remove('mark')

In [64]:
X_train_in = X_all[X_all.part == 'train']
X_val = X_all[X_all.part == 'val']

In [65]:
allmarks = X_train_in['mark']
xtrain = X_train_in[colsss]

In [64]:
from sklearn.model_selection import train_test_split

In [65]:
X_train, X_test, y_train, y_test = train_test_split(X_train_in,X_train_in["mark"],  shuffle=True, test_size=0.2)

In [66]:
X_train["STD_ID"]= X_train["STD_ID"].astype(int)
X_test["STD_ID"]= X_test["STD_ID"].astype(int)

/home/ec2-user/anaconda3/lib/python3.7/site- packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas- docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/ec2-user/anaconda3/lib/python3.7/site- packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas- docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [66]:
hm = X_val[colsss]

In [130]:
ntrain = xtrain.shape[0]
ntest = hm.shape[0]
SEED = 0 # for reproducibility
NFOLDS = 5 # set folds for out-of-fold prediction
kf = KFold(n_splits= NFOLDS, random_state=7)

def get_oof(clf, x_train, y_train, x_test):
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))
    i = 0
    for train_index, test_index in kf.split(x_train):
        x_tr = x_train.iloc[train_index]
        y_tr = y_train.iloc[train_index]
        x_te = x_train.iloc[test_index]

        clf.fit(x_tr, y_tr)
        oof_train[test_index] = clf.predict(x_te)
        oof_test_skf[i, :] = clf.predict(x_test)
        i += 1

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

In [131]:
ooftrainr, ooftestr = get_oof(catreg, xtrain,allmarks,hm)
ooftrainc, ooftestc = get_oof(cb, xtrain,allmarks,hm)

CatBoostError: Bad value for  num_feature[non_default_doc_idx=0,feature_idx=1]="Юриспруденция     ":  Cannot convert  'b'\xd0\xae\xd1\x80\xd0\xb8\xd1\x81\xd0\xbf\xd1\x80\xd1\x83\xd0\ \xb4\xd0\xb5\xd0\xbd\xd1\x86\xd0\xb8\xd1\x8f     '' to float

In [95]:
from sklearn.linear_model import LinearRegression

In [108]:
ensembtrain = np.array([cregpreds.flatten(), cbpreds.flatten()])

In [109]:
lr = LinearRegression()

In [112]:
lr.fit(ensembtrain.T, allmarks)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,  normalize=False)

In [113]:
truepreds = lr.predict( np.array([catreg.predict(hm).flatten(),  cb.predict(hm).flatten()]).T)

In [114]:
truepreds

array([4.0172633 , 4.00983714, 3.2006901 , ..., 3.15833236, 3.20817825,
       3.11105508])

#категориальные фичи

In [68]:
cat_features = ['STD_ID', 'НАПРАВЛЕНИЕ', 'ГОД', 'КУРС', 'СЕМЕСТР',  'АТТЕСТАЦИЯ',
       'ДИСЦИПЛИНА',  'minist', 'level', 'dirgroup',
       'direction', 'studinf_   number', 'studinf_Пол',
       'studinf_Статус', 'studinf_Дата выпуска', 'studinf_Категория обучения',
       'studinf_Форма обучения', 'studinf_Шифр',
       'studinf_направление (специальность)', 'studinf_secret',
       'studinf_Образование', 'studinf_Дата выдачи',
       'studinf_Что именно закончил', 'studinf_Вид ОО', 'studinf_ШИФР_0',
       'studinf_ШИФР_1', 'studinf_ШИФР_2', 'studinf_year', 'studinf_month',
       'studinf_day', 'studinf_Дата выдачи_year', 'studinf_Дата  выдачи_month',
       'studinf_Дата выдачи_day', 'КУРС+СЕМЕСТР', 'studinf_Дата выдачи_year',  'studinf_Дата выдачи_month', 'studinf_Дата выдачи_day', 'is_half_year', 'date'] 

In [69]:
for c in cat_features:
    hm[c] = hm[c].astype('str')
    #xtr[c] = xtr[c].astype('str')
    #xte[c] = xte[c].astype('str')
    X_val[c] = X_val[c].astype('str')
    xtrain[c] = xtrain[c].astype('str')
    X_all[c] = X_all[c].astype('str')

/home/ec2-user/anaconda3/lib/python3.7/site- packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas- docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/ec2-user/anaconda3/lib/python3.7/site- packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas- docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/ec2-user/anaconda3/lib/python3.7/site- packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

# неудачная попытка использовать lightgbm

In [48]:
def labelevthdum(cats, xall):
    xall = xall
    for c in cats:
        dumbxall = pd.get_dummies(xall[c].astype('str'))
        dumbxall.columns = [str(i)+c for i in dumbxall.columns]
        del xall[c]
        xall = xall.merge(dumbxall,how='outer', left_index=True,  right_index=True)
    return xall

In [49]:
def labelevth(cats, xall):
    xall = xall
    for c in cats:
        le = preprocessing.LabelEncoder()
        le.fit(xall[c].astype)
        xall[c] = le.transform(xall[c])
    return xall

In [71]:
cat_features2 = ['НАПРАВЛЕНИЕ',
 'ГОД',
 'КУРС',
 'СЕМЕСТР',
 'АТТЕСТАЦИЯ',
 'minist',
 'level',
 'dirgroup',
 'direction',
 'studinf_Пол',
 'studinf_Статус',
 'studinf_Дата выпуска',
 'studinf_Категория обучения',
 'studinf_Форма обучения',
 'studinf_Шифр',
 'studinf_направление (специальность)',
 'studinf_Образование',
 'studinf_Дата выдачи',
 'studinf_ШИФР_0',
 'studinf_ШИФР_1',
 'studinf_ШИФР_2',
 'studinf_year',
 'studinf_month',
 'studinf_day',
 'studinf_Дата выдачи_year',
 'studinf_Дата выдачи_month',
 'studinf_Дата выдачи_day', 'ДИСЦИПЛИНА', 'studinf_Что именно    закончил']


In [86]:
xall = labelevth(cat_features, X_all.copy())

In [121]:
X_train_inl = xall[xall.part == 'train']
X_vall = xall[xall.part == 'val']
X_trainll, X_testll, y_trainll, y_testll =  train_test_split(X_train_inl,X_train_inl["mark"], shuffle=True, test_size=0.2)

In [122]:
colsss = list(xall.columns)
colsss.remove('part')
colsss.remove('mark')

In [123]:
X_trainll = X_trainll[colsss]
X_testll = X_testll[colsss]

In [124]:
X_vall = X_vall[colsss]

In [125]:
from transliterate import translit, get_available_language_codes


In [126]:
X_trainll.columns = [translit(i, 'ru', reversed=True) for i in X_trainll.columns]
X_testll.columns = [translit(i, 'ru', reversed=True) for i in X_testll.columns]
X_vall.columns = [translit(i, 'ru', reversed=True) for i in X_vall.columns]

In [127]:
X_trainll.columns  = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in  X_trainll.columns]
X_testll.columns  = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in  X_testll.columns]
X_vall.columns  = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in  X_vall.columns]

In [128]:
bst_params = {
    "boosting_type": "gbdt",
    "metric": "mae",
    "objective": "regression",
    "n_jobs": -1,
    "seed": 42,
    "learning_rate": 0.8,
}

fit_params = {
    "num_boost_round": 100_000,
    "early_stopping_rounds": 500,
    "verbose_eval": 50,
}


In [129]:
import lightgbm as lgb

In [130]:
train_set = lgb.Dataset(X_trainll,label=y_trainll)
val_set = lgb.Dataset(X_testll,label=y_testll)

In [131]:
model = lgb.train(
            bst_params,
            train_set,
            valid_sets=[train_set, val_set],
            valid_names=["train", "valid"],
            **fit_params,
        )

Training until validation scores don't improve for 500 rounds
[50]	train's l1: 0.382173	valid's l1: 0.416852
[100]	train's l1: 0.355324	valid's l1: 0.424637
[150]	train's l1: 0.334235	valid's l1: 0.429761
[200]	train's l1: 0.314734	valid's l1: 0.436001
[250]	train's l1: 0.298593	valid's l1: 0.440942
[300]	train's l1: 0.282718	valid's l1: 0.445235
[350]	train's l1: 0.268789	valid's l1: 0.450119
[400]	train's l1: 0.25674	valid's l1: 0.45396
[450]	train's l1: 0.244576	valid's l1: 0.457432
[500]	train's l1: 0.233148	valid's l1: 0.460686
Early stopping, best iteration is:
[39]	train's l1: 0.388405	valid's l1: 0.416199


# Обучаем catboost с loss function: multiclass

In [70]:
import catboost

In [76]:
catboostparamsreg = {
    'od_type': 'Iter',
    'od_wait': 100,
    'learning_rate': 0.07,
    'iterations': 6000,
    'loss_function':'MAE',
    'depth':7,
    'verbose':True,
}

In [77]:
catreg = catboost.CatBoostRegressor(**catboostparamsreg)

In [78]:
catreg.fit(xtrain, allmarks,  cat_features=cat_features)

0:	learn: 0.5842834	total: 180ms	remaining: 18m 1s
1:	learn: 0.5714274	total: 362ms	remaining: 18m 5s
2:	learn: 0.5598170	total: 514ms	remaining: 17m 8s
3:	learn: 0.5484001	total: 683ms	remaining: 17m 4s
4:	learn: 0.5386857	total: 885ms	remaining: 17m 40s
5:	learn: 0.5286669	total: 1.08s	remaining: 18m 3s
6:	learn: 0.5188921	total: 1.28s	remaining: 18m 13s
7:	learn: 0.5100937	total: 1.44s	remaining: 17m 59s
8:	learn: 0.5018779	total: 1.6s	remaining: 17m 47s
9:	learn: 0.4938717	total: 1.77s	remaining: 17m 39s
10:	learn: 0.4868896	total: 1.94s	remaining: 17m 37s
11:	learn: 0.4794621	total: 2.13s	remaining: 17m 42s
12:	learn: 0.4732736	total: 2.3s	remaining: 17m 41s
13:	learn: 0.4676285	total: 2.48s	remaining: 17m 39s
14:	learn: 0.4620816	total: 2.64s	remaining: 17m 35s
15:	learn: 0.4565225	total: 2.82s	remaining: 17m 34s
16:	learn: 0.4512079	total: 3s	remaining: 17m 35s
17:	learn: 0.4463980	total: 3.16s	remaining: 17m 29s
18:	learn: 0.4420969	total: 3.33s	remaining: 17m 28s
19:	learn: 0.

156:	learn: 0.3817812	total: 26s	remaining: 16m 7s
157:	learn: 0.3817725	total: 26.2s	remaining: 16m 6s
158:	learn: 0.3817624	total: 26.3s	remaining: 16m 7s
159:	learn: 0.3817592	total: 26.5s	remaining: 16m 6s
160:	learn: 0.3817311	total: 26.6s	remaining: 16m 6s
161:	learn: 0.3817310	total: 26.7s	remaining: 16m 2s
162:	learn: 0.3817016	total: 26.9s	remaining: 16m 3s
163:	learn: 0.3816654	total: 27.1s	remaining: 16m 3s
164:	learn: 0.3816500	total: 27.2s	remaining: 16m 2s
165:	learn: 0.3816382	total: 27.4s	remaining: 16m 2s
166:	learn: 0.3816362	total: 27.5s	remaining: 16m 1s
167:	learn: 0.3816037	total: 27.7s	remaining: 16m 1s
168:	learn: 0.3815933	total: 27.9s	remaining: 16m 1s
169:	learn: 0.3815220	total: 28s	remaining: 16m 1s
170:	learn: 0.3815154	total: 28.2s	remaining: 16m 1s
171:	learn: 0.3814837	total: 28.3s	remaining: 16m
172:	learn: 0.3814743	total: 28.5s	remaining: 15m 59s
173:	learn: 0.3814113	total: 28.7s	remaining: 15m 59s
174:	learn: 0.3813530	total: 28.8s	remaining: 15m 5

310:	learn: 0.3681346	total: 52.2s	remaining: 15m 55s
311:	learn: 0.3681270	total: 52.4s	remaining: 15m 54s
312:	learn: 0.3681191	total: 52.5s	remaining: 15m 54s
313:	learn: 0.3680921	total: 52.7s	remaining: 15m 54s
314:	learn: 0.3680791	total: 52.9s	remaining: 15m 54s
315:	learn: 0.3680283	total: 53.1s	remaining: 15m 54s
316:	learn: 0.3679933	total: 53.3s	remaining: 15m 55s
317:	learn: 0.3678914	total: 53.5s	remaining: 15m 55s
318:	learn: 0.3678816	total: 53.6s	remaining: 15m 54s
319:	learn: 0.3678081	total: 53.8s	remaining: 15m 55s
320:	learn: 0.3678009	total: 54s	remaining: 15m 54s
321:	learn: 0.3675783	total: 54.1s	remaining: 15m 54s
322:	learn: 0.3675146	total: 54.4s	remaining: 15m 55s
323:	learn: 0.3674495	total: 54.5s	remaining: 15m 55s
324:	learn: 0.3674070	total: 54.7s	remaining: 15m 55s
325:	learn: 0.3673981	total: 54.9s	remaining: 15m 55s
326:	learn: 0.3673551	total: 55.1s	remaining: 15m 55s
327:	learn: 0.3673397	total: 55.2s	remaining: 15m 55s
328:	learn: 0.3673283	total: 5

463:	learn: 0.3615328	total: 1m 18s	remaining: 15m 33s
464:	learn: 0.3615134	total: 1m 18s	remaining: 15m 33s
465:	learn: 0.3614998	total: 1m 18s	remaining: 15m 33s
466:	learn: 0.3614755	total: 1m 18s	remaining: 15m 33s
467:	learn: 0.3613583	total: 1m 18s	remaining: 15m 33s
468:	learn: 0.3613357	total: 1m 19s	remaining: 15m 33s
469:	learn: 0.3612329	total: 1m 19s	remaining: 15m 33s
470:	learn: 0.3612046	total: 1m 19s	remaining: 15m 32s
471:	learn: 0.3611725	total: 1m 19s	remaining: 15m 32s
472:	learn: 0.3611559	total: 1m 19s	remaining: 15m 32s
473:	learn: 0.3611500	total: 1m 19s	remaining: 15m 31s
474:	learn: 0.3611217	total: 1m 20s	remaining: 15m 31s
475:	learn: 0.3611144	total: 1m 20s	remaining: 15m 31s
476:	learn: 0.3611097	total: 1m 20s	remaining: 15m 31s
477:	learn: 0.3610363	total: 1m 20s	remaining: 15m 31s
478:	learn: 0.3609962	total: 1m 20s	remaining: 15m 31s
479:	learn: 0.3609838	total: 1m 20s	remaining: 15m 31s
480:	learn: 0.3609620	total: 1m 21s	remaining: 15m 30s
481:	learn

614:	learn: 0.3578866	total: 1m 43s	remaining: 15m 7s
615:	learn: 0.3578249	total: 1m 43s	remaining: 15m 7s
616:	learn: 0.3578203	total: 1m 43s	remaining: 15m 7s
617:	learn: 0.3577873	total: 1m 44s	remaining: 15m 6s
618:	learn: 0.3577571	total: 1m 44s	remaining: 15m 6s
619:	learn: 0.3577215	total: 1m 44s	remaining: 15m 6s
620:	learn: 0.3577151	total: 1m 44s	remaining: 15m 5s
621:	learn: 0.3577030	total: 1m 44s	remaining: 15m 5s
622:	learn: 0.3576891	total: 1m 44s	remaining: 15m 5s
623:	learn: 0.3576720	total: 1m 45s	remaining: 15m 5s
624:	learn: 0.3576707	total: 1m 45s	remaining: 15m 5s
625:	learn: 0.3575670	total: 1m 45s	remaining: 15m 4s
626:	learn: 0.3575591	total: 1m 45s	remaining: 15m 4s
627:	learn: 0.3575455	total: 1m 45s	remaining: 15m 4s
628:	learn: 0.3575063	total: 1m 45s	remaining: 15m 4s
629:	learn: 0.3574740	total: 1m 46s	remaining: 15m 4s
630:	learn: 0.3574556	total: 1m 46s	remaining: 15m 3s
631:	learn: 0.3574371	total: 1m 46s	remaining: 15m 3s
632:	learn: 0.3574257	total:

...

5394:	learn: 0.3179388	total: 14m 44s	remaining: 1m 39s
5395:	learn: 0.3179325	total: 14m 44s	remaining: 1m 39s
5396:	learn: 0.3179264	total: 14m 44s	remaining: 1m 38s
5397:	learn: 0.3179173	total: 14m 45s	remaining: 1m 38s
5398:	learn: 0.3179140	total: 14m 45s	remaining: 1m 38s
5399:	learn: 0.3179057	total: 14m 45s	remaining: 1m 38s
5400:	learn: 0.3179001	total: 14m 45s	remaining: 1m 38s
5401:	learn: 0.3178969	total: 14m 45s	remaining: 1m 38s
5402:	learn: 0.3178959	total: 14m 45s	remaining: 1m 37s
5403:	learn: 0.3178874	total: 14m 46s	remaining: 1m 37s
5404:	learn: 0.3178797	total: 14m 46s	remaining: 1m 37s
5405:	learn: 0.3178700	total: 14m 46s	remaining: 1m 37s
5406:	learn: 0.3178655	total: 14m 46s	remaining: 1m 37s
5407:	learn: 0.3178644	total: 14m 46s	remaining: 1m 37s
5408:	learn: 0.3178592	total: 14m 46s	remaining: 1m 36s
5409:	learn: 0.3178512	total: 14m 46s	remaining: 1m 36s
5410:	learn: 0.3178485	total: 14m 47s	remaining: 1m 36s
5411:	learn: 0.3178481	total: 14m 47s	remaining:

5542:	learn: 0.3172536	total: 15m 8s	remaining: 1m 14s
5543:	learn: 0.3172530	total: 15m 9s	remaining: 1m 14s
5544:	learn: 0.3172393	total: 15m 9s	remaining: 1m 14s
5545:	learn: 0.3172359	total: 15m 9s	remaining: 1m 14s
5546:	learn: 0.3172342	total: 15m 9s	remaining: 1m 14s
5547:	learn: 0.3172296	total: 15m 9s	remaining: 1m 14s
5548:	learn: 0.3172281	total: 15m 9s	remaining: 1m 13s
5549:	learn: 0.3172250	total: 15m 10s	remaining: 1m 13s
5550:	learn: 0.3172161	total: 15m 10s	remaining: 1m 13s
5551:	learn: 0.3172158	total: 15m 10s	remaining: 1m 13s
5552:	learn: 0.3172145	total: 15m 10s	remaining: 1m 13s
5553:	learn: 0.3172140	total: 15m 10s	remaining: 1m 13s
5554:	learn: 0.3172101	total: 15m 10s	remaining: 1m 12s
5555:	learn: 0.3172031	total: 15m 11s	remaining: 1m 12s
5556:	learn: 0.3171999	total: 15m 11s	remaining: 1m 12s
5557:	learn: 0.3171845	total: 15m 11s	remaining: 1m 12s
5558:	learn: 0.3171811	total: 15m 11s	remaining: 1m 12s
5559:	learn: 0.3171777	total: 15m 11s	remaining: 1m 12s

5693:	learn: 0.3166112	total: 15m 33s	remaining: 50.2s
5694:	learn: 0.3166094	total: 15m 33s	remaining: 50s
5695:	learn: 0.3165961	total: 15m 33s	remaining: 49.8s
5696:	learn: 0.3165957	total: 15m 33s	remaining: 49.7s
5697:	learn: 0.3165915	total: 15m 33s	remaining: 49.5s
5698:	learn: 0.3165902	total: 15m 34s	remaining: 49.3s
5699:	learn: 0.3165689	total: 15m 34s	remaining: 49.2s
5700:	learn: 0.3165659	total: 15m 34s	remaining: 49s
5701:	learn: 0.3165631	total: 15m 34s	remaining: 48.8s
5702:	learn: 0.3165416	total: 15m 34s	remaining: 48.7s
5703:	learn: 0.3165380	total: 15m 34s	remaining: 48.5s
5704:	learn: 0.3165347	total: 15m 35s	remaining: 48.3s
5705:	learn: 0.3165312	total: 15m 35s	remaining: 48.2s
5706:	learn: 0.3165263	total: 15m 35s	remaining: 48s
5707:	learn: 0.3165210	total: 15m 35s	remaining: 47.9s
5708:	learn: 0.3165193	total: 15m 35s	remaining: 47.7s
5709:	learn: 0.3165173	total: 15m 35s	remaining: 47.5s
5710:	learn: 0.3165156	total: 15m 35s	remaining: 47.4s
5711:	learn: 0.3

5843:	learn: 0.3159415	total: 15m 57s	remaining: 25.5s
5844:	learn: 0.3159340	total: 15m 57s	remaining: 25.4s
5845:	learn: 0.3159306	total: 15m 57s	remaining: 25.2s
5846:	learn: 0.3159300	total: 15m 57s	remaining: 25.1s
5847:	learn: 0.3159233	total: 15m 57s	remaining: 24.9s
5848:	learn: 0.3159223	total: 15m 57s	remaining: 24.7s
5849:	learn: 0.3159206	total: 15m 58s	remaining: 24.6s
5850:	learn: 0.3159082	total: 15m 58s	remaining: 24.4s
5851:	learn: 0.3159026	total: 15m 58s	remaining: 24.2s
5852:	learn: 0.3159007	total: 15m 58s	remaining: 24.1s
5853:	learn: 0.3158997	total: 15m 58s	remaining: 23.9s
5854:	learn: 0.3158950	total: 15m 58s	remaining: 23.7s
5855:	learn: 0.3158934	total: 15m 59s	remaining: 23.6s
5856:	learn: 0.3158926	total: 15m 59s	remaining: 23.4s
5857:	learn: 0.3158913	total: 15m 59s	remaining: 23.3s
5858:	learn: 0.3158891	total: 15m 59s	remaining: 23.1s
5859:	learn: 0.3158842	total: 15m 59s	remaining: 22.9s
5860:	learn: 0.3158831	total: 15m 59s	remaining: 22.8s
5861:	lear

5995:	learn: 0.3152790	total: 16m 21s	remaining: 655ms
5996:	learn: 0.3152750	total: 16m 21s	remaining: 491ms
5997:	learn: 0.3152691	total: 16m 22s	remaining: 327ms
5998:	learn: 0.3152654	total: 16m 22s	remaining: 164ms
5999:	learn: 0.3152620	total: 16m 22s	remaining: 0us


In [72]:
catboostparams = {
    'od_type': 'Iter',
    'od_wait': 100,
    'learning_rate': 0.07,
    'iterations': 3000,
    'loss_function':'MultiClass',
    'depth':7,
    'verbose':True,
}

In [73]:
catboostparams2 = {
    'learning_rate': 0.07,
    'iterations': 3200,
    'loss_function':'MultiClass',
    'depth':7,
    'verbose':True,
}

In [117]:
import math
def normal_round(n):
    if n - math.floor(n) < 0.5:
        return math.floor(n)
    return math.ceil(n)

In [74]:
cb = catboost.CatBoostClassifier(**catboostparams2)

In [75]:
cb.fit(xtrain, allmarks,  cat_features=cat_features)

0:	learn: 1.0617966	total: 367ms	remaining: 19m 35s
1:	learn: 1.0311940	total: 771ms	remaining: 20m 33s
2:	learn: 1.0044172	total: 1.15s	remaining: 20m 21s
3:	learn: 0.9785867	total: 1.56s	remaining: 20m 43s
4:	learn: 0.9543021	total: 1.98s	remaining: 21m 3s
5:	learn: 0.9324614	total: 2.39s	remaining: 21m 11s
6:	learn: 0.9128759	total: 2.8s	remaining: 21m 19s
7:	learn: 0.8954806	total: 3.2s	remaining: 21m 15s
8:	learn: 0.8797027	total: 3.58s	remaining: 21m 9s
9:	learn: 0.8653866	total: 4s	remaining: 21m 14s
10:	learn: 0.8526127	total: 4.38s	remaining: 21m 11s
11:	learn: 0.8409196	total: 4.8s	remaining: 21m 15s
12:	learn: 0.8302917	total: 5.2s	remaining: 21m 15s
13:	learn: 0.8199255	total: 5.6s	remaining: 21m 15s
14:	learn: 0.8116167	total: 6.02s	remaining: 21m 18s
15:	learn: 0.8033266	total: 6.44s	remaining: 21m 22s
16:	learn: 0.7957288	total: 6.9s	remaining: 21m 31s
17:	learn: 0.7890036	total: 7.29s	remaining: 21m 28s
18:	learn: 0.7825547	total: 7.73s	remaining: 21m 34s
19:	learn: 0.7

156:	learn: 0.6591698	total: 1m 3s	remaining: 20m 39s
157:	learn: 0.6588959	total: 1m 4s	remaining: 20m 38s
158:	learn: 0.6587257	total: 1m 4s	remaining: 20m 38s
159:	learn: 0.6584474	total: 1m 5s	remaining: 20m 37s
160:	learn: 0.6582679	total: 1m 5s	remaining: 20m 37s
161:	learn: 0.6580997	total: 1m 5s	remaining: 20m 36s
162:	learn: 0.6577308	total: 1m 6s	remaining: 20m 35s
163:	learn: 0.6573848	total: 1m 6s	remaining: 20m 34s
164:	learn: 0.6571601	total: 1m 7s	remaining: 20m 34s
165:	learn: 0.6567860	total: 1m 7s	remaining: 20m 35s
166:	learn: 0.6564959	total: 1m 8s	remaining: 20m 35s
167:	learn: 0.6562243	total: 1m 8s	remaining: 20m 34s
168:	learn: 0.6559807	total: 1m 8s	remaining: 20m 34s
169:	learn: 0.6557222	total: 1m 9s	remaining: 20m 34s
170:	learn: 0.6553821	total: 1m 9s	remaining: 20m 34s
171:	learn: 0.6552117	total: 1m 10s	remaining: 20m 34s
172:	learn: 0.6549288	total: 1m 10s	remaining: 20m 33s
173:	learn: 0.6545534	total: 1m 10s	remaining: 20m 33s
174:	learn: 0.6544010	tot

306:	learn: 0.6303136	total: 2m 4s	remaining: 19m 31s
307:	learn: 0.6301563	total: 2m 4s	remaining: 19m 31s
308:	learn: 0.6300094	total: 2m 5s	remaining: 19m 30s
309:	learn: 0.6299229	total: 2m 5s	remaining: 19m 30s
310:	learn: 0.6297705	total: 2m 5s	remaining: 19m 29s
311:	learn: 0.6296203	total: 2m 6s	remaining: 19m 29s
312:	learn: 0.6295218	total: 2m 6s	remaining: 19m 28s
313:	learn: 0.6294341	total: 2m 7s	remaining: 19m 28s
314:	learn: 0.6292795	total: 2m 7s	remaining: 19m 27s
315:	learn: 0.6291685	total: 2m 7s	remaining: 19m 27s
316:	learn: 0.6289850	total: 2m 8s	remaining: 19m 26s
317:	learn: 0.6288192	total: 2m 8s	remaining: 19m 26s
318:	learn: 0.6287039	total: 2m 9s	remaining: 19m 26s
319:	learn: 0.6285917	total: 2m 9s	remaining: 19m 25s
320:	learn: 0.6285077	total: 2m 9s	remaining: 19m 24s
321:	learn: 0.6283924	total: 2m 10s	remaining: 19m 23s
322:	learn: 0.6283006	total: 2m 10s	remaining: 19m 23s
323:	learn: 0.6282185	total: 2m 10s	remaining: 19m 22s
324:	learn: 0.6280843	tot

457:	learn: 0.6134984	total: 3m 3s	remaining: 18m 20s
458:	learn: 0.6134245	total: 3m 4s	remaining: 18m 20s
459:	learn: 0.6133456	total: 3m 4s	remaining: 18m 19s
460:	learn: 0.6132398	total: 3m 5s	remaining: 18m 19s
461:	learn: 0.6131609	total: 3m 5s	remaining: 18m 19s
462:	learn: 0.6130669	total: 3m 5s	remaining: 18m 18s
463:	learn: 0.6129872	total: 3m 6s	remaining: 18m 18s
464:	learn: 0.6128476	total: 3m 6s	remaining: 18m 17s
465:	learn: 0.6127461	total: 3m 7s	remaining: 18m 17s
466:	learn: 0.6126764	total: 3m 7s	remaining: 18m 17s
467:	learn: 0.6125847	total: 3m 7s	remaining: 18m 16s
468:	learn: 0.6125219	total: 3m 8s	remaining: 18m 16s
469:	learn: 0.6124659	total: 3m 8s	remaining: 18m 15s
470:	learn: 0.6124083	total: 3m 9s	remaining: 18m 15s
471:	learn: 0.6122972	total: 3m 9s	remaining: 18m 15s
472:	learn: 0.6122388	total: 3m 9s	remaining: 18m 14s
473:	learn: 0.6121262	total: 3m 10s	remaining: 18m 14s
474:	learn: 0.6120436	total: 3m 10s	remaining: 18m 14s
475:	learn: 0.6119530	tota

608:	learn: 0.5996344	total: 4m 2s	remaining: 17m 13s
609:	learn: 0.5995366	total: 4m 3s	remaining: 17m 12s
610:	learn: 0.5994379	total: 4m 3s	remaining: 17m 12s
611:	learn: 0.5994041	total: 4m 4s	remaining: 17m 12s
612:	learn: 0.5993083	total: 4m 4s	remaining: 17m 11s
613:	learn: 0.5992196	total: 4m 4s	remaining: 17m 11s
614:	learn: 0.5991347	total: 4m 5s	remaining: 17m 10s
615:	learn: 0.5990580	total: 4m 5s	remaining: 17m 10s
616:	learn: 0.5990049	total: 4m 5s	remaining: 17m 9s
617:	learn: 0.5989322	total: 4m 6s	remaining: 17m 9s
618:	learn: 0.5987991	total: 4m 6s	remaining: 17m 8s
619:	learn: 0.5987062	total: 4m 7s	remaining: 17m 8s
620:	learn: 0.5985942	total: 4m 7s	remaining: 17m 7s
621:	learn: 0.5984899	total: 4m 7s	remaining: 17m 7s
622:	learn: 0.5984038	total: 4m 8s	remaining: 17m 7s
623:	learn: 0.5982462	total: 4m 8s	remaining: 17m 6s
624:	learn: 0.5981864	total: 4m 9s	remaining: 17m 6s
625:	learn: 0.5981479	total: 4m 9s	remaining: 17m 5s
626:	learn: 0.5981016	total: 4m 9s	rem

...

2536:	learn: 0.4800703	total: 16m 50s	remaining: 4m 24s
2537:	learn: 0.4800411	total: 16m 51s	remaining: 4m 23s
2538:	learn: 0.4800276	total: 16m 51s	remaining: 4m 23s
2539:	learn: 0.4799833	total: 16m 51s	remaining: 4m 22s
2540:	learn: 0.4799186	total: 16m 52s	remaining: 4m 22s
2541:	learn: 0.4799110	total: 16m 52s	remaining: 4m 22s
2542:	learn: 0.4798410	total: 16m 53s	remaining: 4m 21s
2543:	learn: 0.4797899	total: 16m 53s	remaining: 4m 21s
2544:	learn: 0.4797270	total: 16m 54s	remaining: 4m 20s
2545:	learn: 0.4796580	total: 16m 54s	remaining: 4m 20s
2546:	learn: 0.4796242	total: 16m 54s	remaining: 4m 20s
2547:	learn: 0.4795826	total: 16m 55s	remaining: 4m 19s
2548:	learn: 0.4795464	total: 16m 55s	remaining: 4m 19s
2549:	learn: 0.4795107	total: 16m 56s	remaining: 4m 19s
2550:	learn: 0.4794612	total: 16m 56s	remaining: 4m 18s
2551:	learn: 0.4794120	total: 16m 56s	remaining: 4m 18s
2552:	learn: 0.4793356	total: 16m 57s	remaining: 4m 17s
2553:	learn: 0.4792938	total: 16m 57s	remaining:

2684:	learn: 0.4726429	total: 17m 50s	remaining: 3m 25s
2685:	learn: 0.4725849	total: 17m 50s	remaining: 3m 24s
2686:	learn: 0.4725147	total: 17m 50s	remaining: 3m 24s
2687:	learn: 0.4724412	total: 17m 51s	remaining: 3m 24s
2688:	learn: 0.4724183	total: 17m 51s	remaining: 3m 23s
2689:	learn: 0.4723500	total: 17m 52s	remaining: 3m 23s
2690:	learn: 0.4723024	total: 17m 52s	remaining: 3m 22s
2691:	learn: 0.4722506	total: 17m 52s	remaining: 3m 22s
2692:	learn: 0.4722029	total: 17m 53s	remaining: 3m 22s
2693:	learn: 0.4721897	total: 17m 53s	remaining: 3m 21s
2694:	learn: 0.4721616	total: 17m 54s	remaining: 3m 21s
2695:	learn: 0.4721063	total: 17m 54s	remaining: 3m 20s
2696:	learn: 0.4720415	total: 17m 54s	remaining: 3m 20s
2697:	learn: 0.4719694	total: 17m 55s	remaining: 3m 20s
2698:	learn: 0.4719570	total: 17m 55s	remaining: 3m 19s
2699:	learn: 0.4719120	total: 17m 55s	remaining: 3m 19s
2700:	learn: 0.4718729	total: 17m 56s	remaining: 3m 18s
2701:	learn: 0.4718040	total: 17m 56s	remaining:

2832:	learn: 0.4652218	total: 18m 48s	remaining: 2m 26s
2833:	learn: 0.4651780	total: 18m 48s	remaining: 2m 25s
2834:	learn: 0.4651306	total: 18m 48s	remaining: 2m 25s
2835:	learn: 0.4651065	total: 18m 49s	remaining: 2m 24s
2836:	learn: 0.4650618	total: 18m 49s	remaining: 2m 24s
2837:	learn: 0.4650210	total: 18m 50s	remaining: 2m 24s
2838:	learn: 0.4649776	total: 18m 50s	remaining: 2m 23s
2839:	learn: 0.4649532	total: 18m 50s	remaining: 2m 23s
2840:	learn: 0.4648950	total: 18m 51s	remaining: 2m 22s
2841:	learn: 0.4648557	total: 18m 51s	remaining: 2m 22s
2842:	learn: 0.4647966	total: 18m 51s	remaining: 2m 22s
2843:	learn: 0.4646817	total: 18m 52s	remaining: 2m 21s
2844:	learn: 0.4646441	total: 18m 52s	remaining: 2m 21s
2845:	learn: 0.4645961	total: 18m 53s	remaining: 2m 20s
2846:	learn: 0.4645491	total: 18m 53s	remaining: 2m 20s
2847:	learn: 0.4644956	total: 18m 53s	remaining: 2m 20s
2848:	learn: 0.4644337	total: 18m 54s	remaining: 2m 19s
2849:	learn: 0.4643842	total: 18m 54s	remaining:

2980:	learn: 0.4583459	total: 19m 45s	remaining: 1m 27s
2981:	learn: 0.4583100	total: 19m 46s	remaining: 1m 26s
2982:	learn: 0.4582877	total: 19m 46s	remaining: 1m 26s
2983:	learn: 0.4582601	total: 19m 47s	remaining: 1m 25s
2984:	learn: 0.4582156	total: 19m 47s	remaining: 1m 25s
2985:	learn: 0.4581739	total: 19m 47s	remaining: 1m 25s
2986:	learn: 0.4581150	total: 19m 48s	remaining: 1m 24s
2987:	learn: 0.4580607	total: 19m 48s	remaining: 1m 24s
2988:	learn: 0.4579990	total: 19m 49s	remaining: 1m 23s
2989:	learn: 0.4579553	total: 19m 49s	remaining: 1m 23s
2990:	learn: 0.4579241	total: 19m 49s	remaining: 1m 23s
2991:	learn: 0.4578621	total: 19m 50s	remaining: 1m 22s
2992:	learn: 0.4578059	total: 19m 50s	remaining: 1m 22s
2993:	learn: 0.4577934	total: 19m 51s	remaining: 1m 21s
2994:	learn: 0.4577639	total: 19m 51s	remaining: 1m 21s
2995:	learn: 0.4577277	total: 19m 51s	remaining: 1m 21s
2996:	learn: 0.4576501	total: 19m 52s	remaining: 1m 20s
2997:	learn: 0.4576032	total: 19m 52s	remaining:

3130:	learn: 0.4513127	total: 20m 44s	remaining: 27.4s
3131:	learn: 0.4512706	total: 20m 45s	remaining: 27s
3132:	learn: 0.4512028	total: 20m 45s	remaining: 26.6s
3133:	learn: 0.4511667	total: 20m 46s	remaining: 26.2s
3134:	learn: 0.4511123	total: 20m 46s	remaining: 25.8s
3135:	learn: 0.4510844	total: 20m 46s	remaining: 25.4s
3136:	learn: 0.4510637	total: 20m 47s	remaining: 25s
3137:	learn: 0.4510072	total: 20m 47s	remaining: 24.6s
3138:	learn: 0.4509829	total: 20m 47s	remaining: 24.3s
3139:	learn: 0.4509321	total: 20m 48s	remaining: 23.9s
3140:	learn: 0.4508590	total: 20m 48s	remaining: 23.5s
3141:	learn: 0.4508059	total: 20m 49s	remaining: 23.1s
3142:	learn: 0.4507705	total: 20m 49s	remaining: 22.7s
3143:	learn: 0.4507134	total: 20m 49s	remaining: 22.3s
3144:	learn: 0.4506507	total: 20m 50s	remaining: 21.9s
3145:	learn: 0.4506253	total: 20m 50s	remaining: 21.5s
3146:	learn: 0.4505615	total: 20m 51s	remaining: 21.1s
3147:	learn: 0.4505222	total: 20m 51s	remaining: 20.7s
3148:	learn: 0

In [126]:
y_pred = pd.read_csv("sample_submission.csv",index_col=0)
y_pred["mark"] = cb.predict(hm)

In [121]:
y_pred['mark'] = y_pred['mark'].apply(normal_round)

In [127]:
y_pred

,mark
index,
56065,4.0
304637,4.0
177437,3.0
222396,4.0
271996,4.0
...,...
49838,5.0
273753,4.0
35426,3.0


In [122]:
y_pred

,mark
index,
56065,4
304637,4
177437,3
222396,4
271996,4
...,...
49838,5
273753,4
35426,3


In [128]:
y_pred.to_csv("submissions/normal2.csv")

In [143]:
sum(np.array(pd.read_csv("submissions/multiclassoverfit.csv")['mark'])-   np.array(y_pred["mark"]))

32.0